# Example with Simulated Data in non-BIDS format

Here our data is saved in examples/data/simulated_Data/data.mat

From nm_IO we use the loadmat function to load this data, and then we can see that the channel names, as well as the bad channels and the sampling frequency are also specified.

The scrip that generates the data is examples/createSimulatedData.py

In [1]:
import os
from py_neuromodulation import (
    nm_define_nmchannels,
    nm_IO,
    nm_stream_offline
)

Set up the necessary paths to the data.

In [2]:
SCRIPT_PATH = os.path.dirname(os.path.abspath(''))
DATA_PATH = os.path.join(SCRIPT_PATH,'examples','data','simulated_Data','data.mat')

PATH_OUT = os.path.join(SCRIPT_PATH,'examples','data','simulated_Data')

In [3]:
data_dict = nm_IO.loadmat(DATA_PATH)

In [4]:
data = data_dict['data'] # voltage values for our simulated data
channels = data_dict['channels'] # array with all channel names
bad = data_dict['bad'] # bad channels
sfreq = data_dict['sfreq']  # sampling frequency

The function set_settings is gonna be needed below, so we can read the settings for the feature estimation.

In [5]:
def set_settings(settings: dict):

    """
    Author: Timon Merk
    Original one can be found on https://github.com/timonmerk/TRD_pynm/blob/main/get_features_from_raw.py
    """

    settings["features"]["fft"] = True
    settings["features"]["fooof"] = False#True
    settings["features"]["return_raw"] = False#True
    settings["features"]["raw_hjorth"] = False#True
    settings["features"]["sharpwave_analysis"] = False#True
    settings["features"]["nolds"] = False#True
    settings["features"]["bursts"] = False#True
    settings["features"]["coherence"] = False#True

    settings["preprocessing"]["re_referencing"] = False
    settings["preprocessing"]["preprocessing_order"] = [
        "raw_resampling",
        "notch_filter",
    ]

    settings["postprocessing"]["feature_normalization"] = True
    settings["postprocessing"]["project_cortex"] = False
    settings["postprocessing"]["project_subcortex"] = False

    settings["fooof"]["periodic"]["center_frequency"] = False
    settings["fooof"]["periodic"]["band_width"] = False
    settings["fooof"]["periodic"]["height_over_ap"] = False

    for key in list(
        settings["sharpwave_analysis_settings"]["sharpwave_features"].keys()
    ):
        settings["sharpwave_analysis_settings"]["sharpwave_features"][
            key
        ] = True
    settings["sharpwave_analysis_settings"]["sharpwave_features"][
        "peak_left"
    ] = False
    settings["sharpwave_analysis_settings"]["sharpwave_features"][
        "peak_right"
    ] = False
    settings["sharpwave_analysis_settings"]["sharpwave_features"][
        "trough"
    ] = False
    settings["sharpwave_analysis_settings"][
        "apply_estimator_between_peaks_and_troughs"
    ] = True

    settings["sharpwave_analysis_settings"]["estimator"]["max"] = [
        "prominence",
        "sharpness",
    ]
    settings["sharpwave_analysis_settings"]["estimator"]["mean"] = [
        "width",
        "interval",
        "decay_time",
        "rise_time",
        "rise_steepness",
        "decay_steepness",
        "slope_ratio",
    ]

    settings["coherence"]["channels"] = [
        ["Cg25L01", "Cg25L03"],
        ["Cg25R01", "Cg25R03"],
        ["Cg25L01", "Cg25R03"],
        ["Cg25R01", "Cg25L03"],
    ]

    settings["coherence"]["frequency_bands"] = ["high beta", "low gamma"]
    settings["coherence"]["method"]["coh"] = True
    settings["coherence"]["method"]["icoh"] = True

    settings["nolds_features"]["sample_entropy"] = False
    settings["nolds_features"]["correlation_dimension"] = False
    settings["nolds_features"]["lyapunov_exponent"] = False
    settings["nolds_features"]["hurst_exponent"] = True
    settings["nolds_features"]["detrended_fluctutaion_analysis"] = False
    settings["nolds_features"]["data"]["raw"] = True
    settings["nolds_features"]["data"]["frequency_bands"] = [
        "low beta",
        "high beta",
        "low gamma",
        "HFA",
    ]

    return settings

In order to give the channel names to the Stream class, we need them to be set with
```nm_define_nmchannels.set_channels```.
We also set the type to be "seeg", but please remember this is actually simulated data.

In [6]:
ch_names = list(channels)
ch_types = ["seeg" for _ in range(len(ch_names))]

nm_channels = nm_define_nmchannels.set_channels(
    ch_names=ch_names,
    ch_types=ch_types,
    reference=None,
    bads=None,
    used_types=["seeg"],
)

stream = nm_stream_offline.Stream(
    settings=None,
    nm_channels=nm_channels,
    verbose=True,
)

stream.set_settings_fast_compute()

stream.settings = set_settings(stream.settings)

stream.init_stream(
    sfreq=sfreq,
    line_noise=50,
)

No Error occurred when testing the settings.
No data specified. Sanity checks related to the length of the signal relative to the filter order will be skipped.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 7.50 Hz
- Upper transition bandwidth: 7.50 Hz
- Filter length: 999 samples (0.999 sec)



## Feature Estimation
The feature estimation step is, then, as straightforward as in example_BIDS.ipynb.

In [7]:
stream.run(
    data=data,
    folder_name="simulated_Data_results",
    out_path_root=PATH_OUT,
)

Last batch took: 0.01 seconds
1.0 seconds of data processed
Last batch took: 0.01 seconds
1.1 seconds of data processed
Last batch took: 0.01 seconds
1.2 seconds of data processed
Last batch took: 0.01 seconds
1.3 seconds of data processed
Last batch took: 0.01 seconds
1.4 seconds of data processed
Last batch took: 0.01 seconds
1.5 seconds of data processed
Last batch took: 0.01 seconds
1.6 seconds of data processed
Last batch took: 0.01 seconds
1.7 seconds of data processed
Last batch took: 0.01 seconds
1.8 seconds of data processed
Last batch took: 0.01 seconds
1.9 seconds of data processed
Last batch took: 0.01 seconds
2.0 seconds of data processed
Last batch took: 0.01 seconds
2.1 seconds of data processed
Last batch took: 0.01 seconds
2.2 seconds of data processed
Last batch took: 0.01 seconds
2.3 seconds of data processed
Last batch took: 0.01 seconds
2.4 seconds of data processed
Last batch took: 0.01 seconds
2.5 seconds of data processed
Last batch took: 0.01 seconds
2.6 second

__Cell below doesn't make sense because I can't plot_target_averaged_channel if there are no targets and no labels.
QUESTION:
Are the things above what you had in mind for this script? Is there anything missing?__

feature_reader = nm_analysis.Feature_Reader(
    feature_dir=PATH_OUT, feature_file="simulated_Data_results"
)

# plot for a single channel
ch_used = feature_reader.nm_channels.query(
    '(type=="seeg") and (used == 1)'
).iloc[0]["name"]

feature_used = (
    "stft" if feature_reader.settings["features"]["stft"] else "fft"
)

feature_reader.plot_target_averaged_channel(
    ch=ch_used,
    list_feature_keywords=[feature_used],
    epoch_len=4,   # Length of epoch in seconds
    threshold=0.5, # Threshold to be used for identifying events
)